In [ ]:
import numpy as np
import matplotlib.pyplot as plt





knn score: 0.801507537688
svm score: 0.809603573423
tree score: 0.721105527638
bayes score: 0.796761585706


In [2]:
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.WARNING)
logging.root.level = logging.WARNING
import os
import jieba
import codecs
import time
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

start = time.clock()
def read_documents(file):
    documents = []
    walk = os.walk(file)
    for root ,dirs, files in walk:
        for name in files:
            filename = os.path.join(root, name)
            try:
                f = open(filename)
                #f = codecs.open(filename,'r','utf-8','ignore')
                text = f.read()
            except:
                f = codecs.open(filename,'r','gbk','ignore')    
                text = f.read()
            finally:
                f.close()

            documents.append(text)
    return documents
#documents = news_dataset.data
documents = read_documents('data')
print ("In the dataset there are", len(documents), "textual documents")
#print ("And this is the first one:\n", documents[0])
        
def load_stopwords():
    f_stop = open('stopwords.txt')
    try:
        f_stop_text = f_stop.read()

    finally:
        f_stop.close()

    f_stop_seg_list = f_stop_text.split('\n')
    return f_stop_seg_list

def tokenize(text):
    stopwords = load_stopwords()    
    return [token for token in jieba.cut(text) if token.strip() not in stopwords and len(token.strip()) > 1]

#print ("After the tokenizer, the previous document becomes:\n", tokenize(documents[0]))


processed_docs = [tokenize(doc) for doc in documents]
word_count_dict = gensim.corpora.Dictionary(processed_docs)
print ("In the corpus there are", len(word_count_dict), "unique tokens")

#word_count_dict.filter_extremes(no_below=20, no_above=0.1)
#print ("After filtering, in the corpus there are only", len(word_count_dict), "unique tokens")

bag_of_words_corpus = [word_count_dict.doc2bow(pdoc) for pdoc in processed_docs]

#bow_doc1 = bag_of_words_corpus[0]
#print ("Bag of words representation of the first document (tuples are composed by token_id and multiplicity):\n", bow_doc1)
#print
#for i in range(5):
#    print ("In the document, topic_id {} (word \"{}\") appears {} time[s]".format(bow_doc1[i][0], word_count_dict[bow_doc1[i][0]], bow_doc1[i][1]))
#print ("...")



end = time.clock()
print('Running time: %s Seconds'%(end-start))
    

In the dataset there are 100 textual documents
In the corpus there are 11490 unique tokens
Running time: 3.0731047387125727 Seconds


In [ ]:
# LDA mono-core
lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=100,iterations=1000, id2word=word_count_dict, passes=5,minimum_probability=0)

# LDA multicore (in this configuration, defaulty, uses n_cores-1)
# lda_model = gensim.models.LdaMulticore(bag_of_words_corpus, num_topics=10, id2word=word_count_dict, passes=5)

#_ = lda_model.print_topics(-1)



In [ ]:
start = time.clock()
#for index, score in sorted(lda_model[bag_of_words_corpus[0]], key=lambda tup: -1*tup[1]):
#    print ("Score: {}\t Index: {}\t \nTopic: {}".format(score, index, lda_model.print_topic(index, 5)))


unseen_documents = read_documents('test')
print ("In the test dataset there are", len(unseen_documents), "unseen textual documents")

processed_docs = [tokenize(doc) for doc in unseen_documents]
#word_count_dict = gensim.corpora.Dictionary(processed_docs)
#print ("In the corpus there are", len(word_count_dict), "unique tokens")

#word_count_dict.filter_extremes(no_below=20, no_above=0.1)
#print ("After filtering, in the corpus there are only", len(word_count_dict), "unique tokens")


bow_vector = [word_count_dict.doc2bow(pdoc) for pdoc in processed_docs]


end = time.clock()
print('Running time: %s Seconds'%(end-start)) 

In [3]:
lper = []
topiclist=[10,50,100,150,200]
for ntopic in topiclist:
    start = time.clock()
    lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=ntopic, iterations=1000,id2word=word_count_dict, passes=5)
    lper.append(lda_model.log_perplexity(bag_of_words_corpus))
    end = time.clock()
    print('Running time: %s Seconds'%(end-start))
print(lper)
# LDA multicore (in this configuration, defaulty, uses n_cores-1)
# lda_model = gensim.models.LdaMulticore(bag_of_words_corpus, num_topics=10, id2word=word_count_dict, passes=5)

Running time: 11.32786949851743 Seconds
Running time: 14.100854407528573 Seconds


Running time: 16.646045065655926 Seconds


Running time: 17.530466260282992 Seconds


Running time: 19.897350404789563 Seconds
[-10.369866605991547, -10.130334107830127, -10.443193078925614, -10.821319194182648, -11.26401110775145]


In [4]:
# LDA mono-core

lper2 = []
iteration =[50,100,500,1000,2000]
for iternum in iteration:
    start = time.clock()
    lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=100, iterations=iternum,id2word=word_count_dict, passes=5)
    lper2.append(lda_model.log_perplexity(bag_of_words_corpus))
    end = time.clock()
    print('Running time: %s Seconds'%(end-start))
print(lper2)

Running time: 16.166576832021065 Seconds
Running time: 16.225584031245262 Seconds


Running time: 20.815404377006644 Seconds


Running time: 24.928525421425775 Seconds


Running time: 27.505073365416365 Seconds
[-10.449027262906808, -10.212015838726003, -9.8670069794212605, -9.9090898258201268, -9.9135197508951673]


In [3]:
import os
import jieba
import codecs
from gensim import models
from gensim import corpora,similarities

def load_stopwords():
    f_stop = open('stopwords.txt')
    try:
        f_stop_text = f_stop.read()

    finally:
        f_stop.close()

    f_stop_seg_list = f_stop_text.split('\n')
    return f_stop_seg_list

train_set = []
walk = os.walk('data')
for root ,dirs, files in walk:
    for name in files:
        filename = os.path.join(root, name)
        try:
            f = open(filename)
            #f = codecs.open(filename,'r','utf-8','ignore')
            raw = f.read()
        except:
            f = codecs.open(filename,'r','gbk','ignore')    
            raw = f.read()
        finally:
            f.close()
        word_list = list(jieba.cut(raw, cut_all = False))
        train_set.append(word_list)
        

    
dic = corpora.Dictionary(train_set)
corpus = [dic.doc2bow(text) for text in train_set]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
lda = models.LdaModel(corpus_tfidf, id2word = dic, num_topics = 10,passes = 5)
corpus_lda = lda[corpus_tfidf]


In [14]:
x = [10,50,100,150,200]
y = [-8.4073949386240194, -8.4535082181200831, -8.4753371336751471, -8.4235092637521234, -8.4105535283007384]
print("num_topics: ",x)
print("log_perplexity: ",y)

num_topics:  [10, 50, 100, 150, 200]
log_perplexity:  [-8.40739493862402, -8.453508218120083, -8.475337133675147, -8.423509263752123, -8.410553528300738]


In [23]:
import numpy as np

doc_type = np.arange(10746)

doc_type[0:1194] = 1
print(doc_type[1203])

1203


In [35]:
myList1=[0]*10
myList1.extend([1]*10)
for i in range(9):
    myList.extend([i]*10)
myList1



[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [30]:
help(list)

Help on class list in module builtins:

class list(object)
 |  list() -> new empty list
 |  list(iterable) -> new list initialized from iterable's items
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __l

In [24]:
help(doc_type)

Help on ndarray object:

class ndarray(builtins.object)
 |  ndarray(shape, dtype=float, buffer=None, offset=0,
 |          strides=None, order=None)
 |  
 |  An array object represents a multidimensional, homogeneous array
 |  of fixed-size items.  An associated data-type object describes the
 |  format of each element in the array (its byte-order, how many bytes it
 |  occupies in memory, whether it is an integer, a floating point number,
 |  or something else, etc.)
 |  
 |  Arrays should be constructed using `array`, `zeros` or `empty` (refer
 |  to the See Also section below).  The parameters given here refer to
 |  a low-level method (`ndarray(...)`) for instantiating an array.
 |  
 |  For more information, refer to the `numpy` module and examine the
 |  the methods and attributes of an array.
 |  
 |  Parameters
 |  ----------
 |  (for the __new__ method; see Notes below)
 |  
 |  shape : tuple of ints
 |      Shape of created array.
 |  dtype : data-type, optional
 |      Any o

In [4]:
_ = lda.print_topics(-1)
_

[(0,
  '0.000*利苑 + 0.000*辽宁队 + 0.000*实验室 + 0.000*联想 + 0.000*医药 + 0.000*降价 + 0.000*VS + 0.000*作战 + 0.000*本场 + 0.000*初盘'),
 (1,
  '0.001*０ + 0.000*雷克萨斯 + 0.000*奇瑞 + 0.000*２ + 0.000*元老 + 0.000*１ + 0.000*围棋 + 0.000*销量 + 0.000*人口 + 0.000*止痛药'),
 (2,
  '0.001*& + 0.001*nbsp + 0.001*; + 0.000*耿大勇 + 0.000*新飞 + 0.000*戴尔公司 + 0.000*坦克 + 0.000*邮票 + 0.000*股骨头 + 0.000*宁波'),
 (3,
  '0.001*\x00 + 0.001*nbsp + 0.001*& + 0.000*新浪 + 0.000*黑车 + 0.000*; + 0.000*游客 + 0.000*机票 + 0.000*联想 + 0.000*医务人员'),
 (4,
  '0.001*袁 + 0.001*旅游 + 0.001*游客 + 0.001*小 + 0.001*你 + 0.001*我 + 0.001*公司 + 0.001*东莞 + 0.001*他 + 0.001*支付'),
 (5,
  '0.001*新浪 + 0.001*\r\n + 0.000*易趣网 + 0.000*股东 + 0.000*H股 + 0.000*海上 + 0.000*升值 + 0.000*拦截 + 0.000*自卫队 + 0.000*作战'),
 (6,
  '0.001*认证 + 0.000*车贷险 + 0.000*词汇 + 0.000*保险公司 + 0.000*肇庆 + 0.000*牙膏 + 0.000*还款 + 0.000*护理 + 0.000*句子 + 0.000*走廊'),
 (7,
  '0.001*女兵 + 0.000*兵器 + 0.000*考生 + 0.000*低腰裤 + 0.000*志愿 + 0.000*三九生化 + 0.000*银行 + 0.000*我 + 0.000*词汇 + 0.000*家教'),
 (8,
  '0.001*考古 + 0.001*  + 0.001

In [8]:
for index, score in sorted(lda[corpus_tfidf[0]], key=lambda tup: -1*tup[1]):
    print ("Score: {}\t Index:{}\t Topic: {}".format(score, index, lda.print_topic(index, 10)))

Score: 0.6636686621953762	 Index:2	 Topic: 0.001*& + 0.001*nbsp + 0.001*; + 0.000*耿大勇 + 0.000*新飞 + 0.000*戴尔公司 + 0.000*坦克 + 0.000*邮票 + 0.000*股骨头 + 0.000*宁波
Score: 0.273083399085053	 Index:4	 Topic: 0.001*袁 + 0.001*旅游 + 0.001*游客 + 0.001*小 + 0.001*你 + 0.001*我 + 0.001*公司 + 0.001*东莞 + 0.001*他 + 0.001*支付


In [10]:
for index, score in lda[corpus_tfidf[0]]:
    print ("Score: {}\t Index:{}\t Topic: {}\n".format(score, index, lda.print_topic(index, 10)))

Score: 0.6635615306801939	 Index:2	 Topic: 0.001*& + 0.001*nbsp + 0.001*; + 0.000*耿大勇 + 0.000*新飞 + 0.000*戴尔公司 + 0.000*坦克 + 0.000*邮票 + 0.000*股骨头 + 0.000*宁波

Score: 0.27319036084839016	 Index:4	 Topic: 0.001*袁 + 0.001*旅游 + 0.001*游客 + 0.001*小 + 0.001*你 + 0.001*我 + 0.001*公司 + 0.001*东莞 + 0.001*他 + 0.001*支付



In [7]:
#     >>> lda = LdaModel(corpus, num_topics=100)  # train model
lda[corpus_tfidf] # get topic probability distribution for a document
#lda.update(corpus2) # update the LDA model with additional documents
#print(lda[doc_bow])
#lda = LdaModel(corpus, num_topics=50, alpha='auto', eval_every=5)  # train asymmetric alpha from data

In [74]:
unseen_document = "In my spare time I either play badmington or drive my car"
print ("The unseen document is composed by the following text:", unseen_document)
print

bow_vector = word_count_dict.doc2bow(tokenize(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print ("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

The unseen document is composed by the following text: In my spare time I either play badmington or drive my car


NameError: name 'word_count_dict' is not defined

In [76]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
__title__ = 'topic model - build lda - 20news dataset'
__author__ = 'pi'
__mtime__ = '12/26/2014-026'
# code is far away from bugs with the god animal protecting
    I love animals. They taste delicious.
              ┏┓      ┏┓
            ┏┛┻━━━┛┻┓
            ┃      ☃      ┃
            ┃  ┳┛  ┗┳  ┃
            ┃      ┻      ┃
            ┗━┓      ┏━┛
                ┃      ┗━━━┓
                ┃  神兽保佑    ┣┓
                ┃　永无BUG！   ┏┛
                ┗┓┓┏━┳┓┏┛
                  ┃┫┫  ┃┫┫
                  ┗┻┛  ┗┻┛
"""
#from Colors import *
from collections import defaultdict
import re
import datetime
from sklearn import datasets
import nltk
from gensim import corpora
from gensim import models
import numpy as np
from scipy import spatial
from CorePyPro.Fun.TimeStump import totalTime


def load_texts(dataset_type='train', groups=None):
    """
    load datasets to bytes list
    :return:train_dataset_bunch.data bytes list
    """
    if groups == 'small':
        groups = ['comp.graphics', 'comp.os.ms-windows.misc']  # 仅用于小数据测试时用, #1368
    elif groups == 'medium':
        groups = ['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.ma c.hardware',
                  'comp.windows.x', 'sci.space']  # 中量数据时用    #3414
    train_dataset_bunch = datasets.load_mlcomp('20news-18828', dataset_type, mlcomp_root='./datasets',
                                               categories=groups)  # 13180
    return train_dataset_bunch.data


def preprocess_texts(texts, test_doc_id=1):
    """
    texts preprocessing
    :param texts: bytes list
    :return:bytes list
    """
    texts = [t.decode(errors='ignore') for t in texts]  # bytes2str
    # print(REDH, 'original texts[%d]: ' % test_doc_id, DEFAULT, '\n',texts[test_doc_id])
    # split_texts = [t.lower().split() for t in texts]
    # print(REDH, 'split texts[%d]: #%d' % (test_doc_id, len(split_texts)), DEFAULT, '\n',split_texts[test_doc_id])


    # lower str & split str 2 word list with sep=... & delete None
    SEPS = '[\s()-/,:.?!]\s*'
    texts = [re.split(SEPS, t.lower()) for t in texts]
    for t in texts:
        while '' in t:
            t.remove('')
    # print(REDH, 'texts[%d] lower & split(seps= %s) & delete None: #%d' % (test_doc_id, SEPS, len(texts[test_doc_id])), DEFAULT, '\n',texts[test_doc_id])


    # nltk.download()   #then choose the corpus.stopwords
    stopwords = set(nltk.corpus.stopwords.words('english'))  # #127
    stopwords.update(['from', 'subject', 'writes'])  # #129
    word_usage = defaultdict(int)
    for t in texts:
        for w in t:
            word_usage[w] += 1
    COMMON_LINE = len(texts) / 10
    too_common_words = [w for w in t if word_usage[w] > COMMON_LINE]  # set(too_common_words)
    # print('too_common_words: #', len(too_common_words), '\n', too_common_words)   #68
    stopwords.update(too_common_words)
    # print('stopwords: #', len(stopwords), '\n', stopwords)  #   #147

    english_stemmer = nltk.SnowballStemmer('english')
    MIN_WORD_LEN = 3  # 4
    texts = [[english_stemmer.stem(w) for w in t if
              not set(w) & set('@+>0123456789*') and w not in stopwords and len(w) >= MIN_WORD_LEN] for t in
             texts]  # set('+-.?!()>@0123456789*/')
    # print(REDH, 'texts[%d] delete ^alphanum & stopwords & len<%d & stemmed: #' % (test_doc_id, MIN_WORD_LEN),
    # len(texts[test_doc_id]), DEFAULT, '\n', texts[test_doc_id])
    return texts


def build_corpus(texts):
    """
    build corpora
    :param texts: bytes list
    :return: corpus DirectTextCorpus(corpora.TextCorpus)
    """

    class DirectTextCorpus(corpora.TextCorpus): 
        def get_texts(self):
            return self.input

        def __len__(self):
            return len(self.input)

    corpus = DirectTextCorpus(texts)
    return corpus


def build_id2word(corpus):
    """
    from corpus build id2word=dict
    :param corpus:
    :return:dict = corpus.dictionary
    """
    dict = corpus.dictionary  # gensim.corpora.dictionary.Dictionary
    # print(dict.id2token)
    try:
        dict['anything']
    except:
        pass
        # print("dict.id2token is not {} now")
    # print(dict.id2token)
    return dict


def save_corpus_dict(dict, corpus, dictDir='./LDA/id_word.dict', corpusDir='./LDA/corpus.mm'):
    dict.save(dictDir)
    print(GREENL, 'dict saved into %s successfully ...' % dictDir, DEFAULT)
    corpora.MmCorpus.serialize(corpusDir, corpus)
    print(GREENL, 'corpus saved into %s successfully ...' % corpusDir, DEFAULT)
    # corpus.save(fname='./LDA/corpus.mm')  # stores only the (tiny) iteration object


def load_ldamodel(modelDir='./lda.pkl'):
    model = models.LdaModel.load(fname=modelDir)
    print(GREENL, 'ldamodel load from %s successfully ...' % modelDir, DEFAULT)
    return model


def load_corpus_dict(dictDir='./LDA/id_word.dict', corpusDir='./LDA/corpus.mm'):
    dict = corpora.Dictionary.load(fname=dictDir)
    print(GREENL, 'dict load from %s successfully ...' % dictDir, DEFAULT)
    # dict = corpora.Dictionary.load_from_text('./id_word.txt')
    corpus = corpora.MmCorpus(corpusDir)  # corpora.mmcorpus.MmCorpus
    print(GREENL, 'corpus load from %s successfully ...' % corpusDir, DEFAULT)
    return dict, corpus


def build_doc_word_mat(corpus, model, num_topics):
    """
    build doc_word_mat in topic space
    :param corpus:
    :param model:
    :param num_topics: int
    :return:doc_word_mat np.array (len(topics) * num_topics)
    """
    topics = [model[c] for c in corpus]  # (word_id, weight) list
    doc_word_mat = np.zeros((len(topics), num_topics))
    for doc, topic in enumerate(topics):
        for word_id, weight in topic:
            doc_word_mat[doc, word_id] += weight
    return doc_word_mat


def compute_pairwise_dist(doc_word_mat):
    """
    compute pairwise dist
    :param doc_word_mat: np.array (len(topics) * num_topics)
    :return:pairwise_dist <class 'numpy.ndarray'>
    """
    pairwise_dist = spatial.distance.squareform(spatial.distance.pdist(doc_word_mat))
    max_weight = pairwise_dist.max() + 1
    for i in list(range(len(pairwise_dist))):
        pairwise_dist[i, i] = max_weight
    return pairwise_dist


def closest_texts(corpus, model, num_topics, test_doc_id=1, topn=5):
    """
    find the closest_doc_ids for  doc[test_doc_id]
    :param corpus:
    :param model:
    :param num_topics:
    :param test_doc_id:
    :param topn:
    :return:
    """
    doc_word_mat = build_doc_word_mat(corpus, model, num_topics)
    pairwise_dist = compute_pairwise_dist(doc_word_mat)
    # print(REDH, 'original texts[%d]: ' % test_doc_id, DEFAULT, '\n', original_texts[test_doc_id])
    closest_doc_ids = pairwise_dist[test_doc_id].argsort()
    # return closest_doc_ids[:topn]
    for closest_doc_id in closest_doc_ids[:topn]:
        print(RED, 'closest doc[%d]' % closest_doc_id, DEFAULT, '\n', original_texts[closest_doc_id])


def evaluate_model(model):
    """
    計算模型在test data的Perplexity
    :param model:
    :return:model.log_perplexity float
    """
    test_texts = load_texts(dataset_type='test', groups='small')
    test_texts = preprocess_texts(test_texts)
    test_corpus = build_corpus(test_texts)
    return model.log_perplexity(test_corpus)


def test_num_topics():
    dict, corpus = load_corpus_dict()
    print("#corpus_items:", len(corpus))
    for num_topics in [3, 5, 10, 30, 50, 100, 150, 200, 300]:
        start_time = datetime.datetime.now()
        model = models.LdaModel(corpus, num_topics=num_topics, id2word=dict)
        end_time = datetime.datetime.now()
        print("total running time = ", end_time - start_time)
        print(REDL, 'model.log_perplexity for test_texts with num_topics=%d : ' % num_topics, evaluate_model(model),
              DEFAULT)


def test():
    texts = load_texts(dataset_type='train', groups='small')
    original_texts = texts
    test_doc_id = 1

    # texts = preprocess_texts(texts, test_doc_id=test_doc_id)
    # corpus = build_corpus(texts=texts)  # corpus DirectTextCorpus(corpora.TextCorpus)
    # dict = build_id2word(corpus)
    # save_corpus_dict(dict, corpus)
    dict, corpus = load_corpus_dict()
    # print(len(corpus))

    num_topics = 100
    model = models.LdaModel(corpus, num_topics=num_topics, id2word=dict)  # 每次结果不同
    model.show_topic(0)
    # model.save(fname='./lda.pkl')

    # model = load_ldamodel()
    # closest_texts(corpus, model, num_topics, test_doc_id=1, topn=3)

    print(REDL, 'model.log_perplexity for test_texts', evaluate_model(model), DEFAULT)


if __name__ == '__main__':
    test()
    # test_num_topics()

ImportError: No module named 'CorePyPro'

In [6]:
import os
from os.path import join, getsize
for root, dirs, files in os.walk('Reduced'):
    print(root, "consumes", end="")
    print(sum([getsize(join(root, name)) for name in files]), end="")
    print("bytes in", len(files), "non-directory files")
    if 'CVS' in dirs:
        dirs.remove('CVS')  # don't visit CVS directories

Reduced consumes0bytes in 0 non-directory files
Reduced\C000008 consumes4749812bytes in 1990 non-directory files
Reduced\C000010 consumes3539444bytes in 1990 non-directory files
Reduced\C000013 consumes5078678bytes in 1990 non-directory files
Reduced\C000014 consumes3213181bytes in 1990 non-directory files
Reduced\C000016 consumes4155989bytes in 1990 non-directory files
Reduced\C000020 consumes6791509bytes in 1990 non-directory files
Reduced\C000022 consumes6696483bytes in 1990 non-directory files
Reduced\C000023 consumes11397015bytes in 1990 non-directory files
Reduced\C000024 consumes4961840bytes in 1990 non-directory files


In [40]:
import codecs,sys

f = codecs.open('1107.txt','r','utf-8')
try:
    text = f.read()
    #text =unicode(text,'utf-8')
finally:
    f.close()
print(text)

﻿
跳空过大关 双高仍可期
北 京 首 证

两市周二再次跳空高开并呈极明显的强势上攻态势，股指尾市再创新高并收于次高点，涨跌家数之比显示市场依然保持全线普涨的壮观景象，成交总额更是高达700亿元，较前激增约180亿元或34％左右，去掉权证因素的话，日成交总额也达到590亿元以上，这无疑是近年来的天量。
周二两市之所以继续保持强势上行之势，关键就在于市场的做多热情已经并正在不断提高，涨跌家数之比的变化便充分表明了这一点，且不论涨幅在5％以上的个股比比皆是，仅以涨停家数而言，沪深两市就比周一又一次有所增加。值得注意的是，作为市场的中流砥柱，大盘蓝筹板块内部再次出现有序的轮动格局，两市各自的第一航母即中国石化(行情,论坛)、G中兴(行情,论坛)多数时间内保持强势整固姿态，G宝钢(行情,论坛)、G长电(行情,论坛)、G招行(行情,论坛)和G万科等的短线涨幅也弱于股指，一定程度显示多方主力目前并不想大幅推高股指，而G华能(行情,论坛)、G申能(行情,论坛)和G华侨城(行情,论坛)等的表现相对较强，且G五粮液(行情,论坛)的身影还出现在涨停行列中，尾市深发展等大盘银行股也出现大幅上扬之势，也就是说，目前既有一部分超级航母直接参与推升大盘，也有一部分权重很大的蓝筹龙头品种引而不发，这就在相当大程度上确保了市场近日将会继续强攻。
盘面观察可发现，两市绝大多数个股与板块都已经形成强攻态势，市场的热点由此被完全激活，换言之，市场热点并非以往的以某一类型的个股为主线，而是形成了“满园春色关不?钡木跋螅∑渲校绕渲档靡惶岬氖牵谛矶喔龉沙中耐保ㄖて分值谋硐指欠绻馕尴蓿扔形辶竃GC1在一个月左右的时间内大涨500％以上，也有在一天内暴涨61.59％的万华HXP1！这种效应对市场的影响力无疑是相当大的，很大程度上也表明市场继续强势上攻的可能依然相当大。
从股指运行看，两市大盘周二再次跳空高开高走，日线走势图上由此也产生了连续的第二个上升缺口，其中上证综指的还恰好出现在千五整数和心理大关处，这对市场也会产生很大的积极影响，而且成交额也创出了近阶段的新高，单日成交近600亿元（历史天量的64％左右）也表明增量资金入市热情仍然非常高涨！因此，两市近日很可能将会继续强势上行，“双高”即股价和成交额的新高纪录也将在强攻过程中不断被刷新。 


在本机构所知情的范围内，本机构及财产上的利害

In [ ]:
import codecs
help(codecs)
import os
help(os.walk)

In [1]:
import gensim
help(gensim)

Help on package gensim:

NAME
    gensim

DESCRIPTION
    This package contains interfaces and functionality to compute pair-wise document
    similarities within a corpus of documents.

PACKAGE CONTENTS
    corpora (package)
    interfaces
    matutils
    models (package)
    nosy
    parsing (package)
    scripts (package)
    similarities (package)
    summarization (package)
    test (package)
    utils

CLASSES
    logging.Handler(logging.Filterer)
        NullHandler
    
    class NullHandler(logging.Handler)
     |  For python versions <= 2.6; same as `logging.NullHandler` in 2.7.
     |  
     |  Method resolution order:
     |      NullHandler
     |      logging.Handler
     |      logging.Filterer
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  emit(self, record)
     |  
     |  ----------------------------------------------------------------------
     |  Methods inherited from logging.Handler:
     |  
     |  __init__(self, level=0)


In [11]:
from gensim.test import test_ldamodel

help(test_ldamodel)

Help on module gensim.test.test_ldamodel in gensim.test:

NAME
    gensim.test.test_ldamodel - Automated tests for checking transformation algorithms (the models package).

CLASSES
    unittest.case.TestCase(builtins.object)
        TestLdaModel
            TestLdaMulticore
    
    class TestLdaModel(unittest.case.TestCase)
     |  Method resolution order:
     |      TestLdaModel
     |      unittest.case.TestCase
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  setUp(self)
     |  
     |  testAlpha(self)
     |  
     |  testAlphaAuto(self)
     |  
     |  testEta(self)
     |  
     |  testEtaAuto(self)
     |  
     |  testGetDocumentTopics(self)
     |  
     |  testGetTopicTerms(self)
     |  
     |  testLargeMmap(self)
     |  
     |  testLargeMmapCompressed(self)
     |  
     |  testPasses(self)
     |  
     |  testPersistence(self)
     |  
     |  testPersistenceCompressed(self)
     |  
     |  testPersistenceIgnore(self)
     |  
  

In [3]:
from gensim import models
help(models)

Help on package gensim.models in gensim:

NAME
    gensim.models

DESCRIPTION
    This package contains algorithms for extracting document representations from their raw
    bag-of-word counts.

PACKAGE CONTENTS
    doc2vec
    hdpmodel
    lda_dispatcher
    lda_worker
    ldamodel
    ldamulticore
    logentropy_model
    lsi_dispatcher
    lsi_worker
    lsimodel
    phrases
    rpmodel
    tfidfmodel
    word2vec
    wrappers (package)

CLASSES
    gensim.interfaces.TransformationABC(gensim.utils.SaveLoad)
        VocabTransform
    
    class VocabTransform(gensim.interfaces.TransformationABC)
     |  Remap feature ids to new values.
     |  
     |  Given a mapping between old ids and new ids (some old ids may be missing = these
     |  features are to be discarded), this will wrap a corpus so that iterating over
     |  `VocabTransform[corpus]` returns the same vectors but with the new ids.
     |  
     |  Old features that have no counterpart in the new ids are discarded. This

In [5]:
help(models.ldamodel)

Help on module gensim.models.ldamodel in gensim.models:

NAME
    gensim.models.ldamodel - **For a faster implementation of LDA (parallelized for multicore machines), see** :mod:`gensim.models.ldamulticore`.

DESCRIPTION
    Latent Dirichlet Allocation (LDA) in Python.
    
    This module allows both LDA model estimation from a training corpus and inference of topic
    distribution on new, unseen documents. The model can also be updated with new documents
    for online training.
    
    The core estimation code is based on the `onlineldavb.py` script by M. Hoffman [1]_, see
    **Hoffman, Blei, Bach: Online Learning for Latent Dirichlet Allocation, NIPS 2010.**
    
    The algorithm:
    
    * is **streamed**: training documents may come in sequentially, no random access required,
    * runs in **constant memory** w.r.t. the number of documents: size of the
      training corpus does not affect memory footprint, can process corpora larger than RAM, and
    * is **distributed**: m

In [ ]:
lda = LdaModel(corpus, num_topics=100)  # train model
print(lda[doc_bow]) # get topic probability distribution for a document
lda.update(corpus2) # update the LDA model with additional documents
print(lda[doc_bow])
lda = LdaModel(corpus, num_topics=50, alpha='auto', eval_every=5)  # train asymmetric alpha from data



In [ ]:
#encoding:utf-8
'''
Created on 2015年10月25日

@author: Administrator
'''
import pandas  as pd
import re
import jieba  
import nltk  
import jieba.posseg as pseg  
from gensim import corpora, models, similarities
df=pd.read_csv(u'C:\\Users\\Administrator\\Desktop\\质量记录.csv',encoding='gbk')
cont=df['QUALITYDESC'].map(lambda x:re.sub(ur'客户反应|客户|反映','',x)).map(lambda x:re.sub(r'[a-zA-Z0-9\.]','',x))
#导入自己添加的用户词语
jieba.load_userdict(u'C:\\Users\\Administrator\\Desktop\\分词.txt')

nwordall = []  
for t in cont:  
        words =pseg.cut(t)  
        nword = ['']  
        for w in words:  
            if((w.flag == 'n'or w.flag == 'v' or w.flag == 'a') and len(w.word)>1):  
                nword.append(w.word)  
        nwordall.append(nword) 
# 选择后的词生成字典  
dictionary = corpora.Dictionary(nwordall)#用于生成字典类似与table，Counter模块中count
    #print dictionary.token2id  
# 生成语料库   
corpus = [dictionary.doc2bow(text) for text in nwordall]
 #tfidf加权  
tfidf = models.TfidfModel(corpus)  
# print tfidf.dfsx  
# print tfidf.idf  
corpus_tfidf = tfidf[corpus]

 # 4. 主题模型lda，可用于降维  
#lda流式数据建模计算，每块10000条记录，提取50个主题  
lda = models.ldamodel.LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=50,     update_every=1, chunksize=10000, passes=1)  
#提取前面20个主题
for i in range(0,20):  
    print lda.print_topics(i)[0]  

#lda全部数据建模，提取100个主题  
#lda = models.ldamodel.LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=100, update_every=0, passes=20)  
#利用原模型预测新文本主题  
#doc_lda = lda[corpus_tfidf]

 
#5. word2vec 词向量化，可用于比较词相似度，寻找对应关系，词聚类  
#sentences = models.word2vec.LineSentence(nwordall)  
#size为词向量维度数,windows窗口范围,min_count频数小于5的词忽略,workers是线程数  
model = models.word2vec.Word2Vec(nwordall, size=100, window=5, min_count=5, workers=4)
print model[u'指示灯']  
#向量表示  
sim = model.most_similar(positive=[u'指示灯', u'灯不亮'])  
    #相近词  
for s in sim:  
    print "word:%s,similar:%s " %(s[0],s[1])





In [2]:
import sklearn
from sklearn import svm
help(svm)

Help on package sklearn.svm in sklearn:

NAME
    sklearn.svm - The :mod:`sklearn.svm` module includes Support Vector Machine algorithms.

PACKAGE CONTENTS
    base
    bounds
    classes
    liblinear
    libsvm
    libsvm_sparse
    setup
    tests (package)

CLASSES
    sklearn.base.BaseEstimator(builtins.object)
        sklearn.svm.classes.LinearSVC(sklearn.base.BaseEstimator, sklearn.linear_model.base.LinearClassifierMixin, sklearn.feature_selection.from_model._LearntSelectorMixin, sklearn.linear_model.base.SparseCoefMixin)
    sklearn.base.RegressorMixin(builtins.object)
        sklearn.svm.classes.LinearSVR(sklearn.linear_model.base.LinearModel, sklearn.base.RegressorMixin)
        sklearn.svm.classes.NuSVR(sklearn.svm.base.BaseLibSVM, sklearn.base.RegressorMixin)
        sklearn.svm.classes.SVR(sklearn.svm.base.BaseLibSVM, sklearn.base.RegressorMixin)
    sklearn.feature_selection.from_model._LearntSelectorMixin(sklearn.base.TransformerMixin)
        sklearn.svm.classes.LinearS

In [8]:
from sklearn.svm import tests
help(tests)

Help on package sklearn.svm.tests in sklearn.svm:

NAME
    sklearn.svm.tests

PACKAGE CONTENTS
    test_bounds
    test_sparse
    test_svm

FILE
    d:\360downloads\anaconda3\lib\site-packages\sklearn\svm\tests\__init__.py




In [13]:
from sklearn.svm.tests import test_svm
help(tests.test_svm)

Help on module sklearn.svm.tests.test_svm in sklearn.svm.tests:

NAME
    sklearn.svm.tests.test_svm - Testing for Support Vector Machine module (sklearn.svm)

DESCRIPTION
    TODO: remove hard coded numerical results when possible

FUNCTIONS
    test_auto_weight()
    
    test_bad_input()
    
    test_consistent_proba()
    
    test_crammer_singer_binary()
    
    test_decision_function()
    
    test_dense_liblinear_intercept_handling(classifier=<class 'sklearn.svm.classes.LinearSVC'>)
    
    test_immutable_coef_property()
    
    test_inheritance()
    
    test_liblinear_set_coef()
    
    test_libsvm_iris()
    
    test_libsvm_parameters()
    
    test_linear_svc_convergence_warnings()
    
    test_linear_svc_intercept_scaling()
    
    test_linear_svx_uppercase_loss_penalty()
        # FIXME remove in 0.18
    
    test_linearsvc()
    
    test_linearsvc_crammer_singer()
    
    test_linearsvc_iris()
    
    test_linearsvc_parameters()
    
    test_linearsvc_verb

In [15]:
y_train =[]

In [16]:
for i in range(9):
    y_train.extend([i]*1592)

In [17]:
len(y_train)

14328

In [18]:
y_test = []
for i in range(9):
    y_test.extend([i]*398)


In [19]:
len(y_test)

3582

In [21]:
import gensim.interfaces
help(gensim.interfaces.TransformedCorpus)

Help on class TransformedCorpus in module gensim.interfaces:

class TransformedCorpus(CorpusABC)
 |  Method resolution order:
 |      TransformedCorpus
 |      CorpusABC
 |      gensim.utils.SaveLoad
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getitem__(self, docno)
 |  
 |  __init__(self, obj, corpus, chunksize=None)
 |  
 |  __iter__(self)
 |  
 |  __len__(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from CorpusABC:
 |  
 |  save(self, *args, **kwargs)
 |  
 |  ----------------------------------------------------------------------
 |  Static methods inherited from CorpusABC:
 |  
 |  save_corpus(fname, corpus, id2word=None, metadata=False)
 |      Save an existing `corpus` to disk.
 |      
 |      Some formats also support saving the dictionary (`feature_id->word` mapping),
 |      which can in this case be provided by the optional `id2word` parameter.
 |      
 |      >>> MmCorpus.save_corpus('file

In [ ]:

help(print)